In [ ]:
from atmospheric_explorer.cams_interfaces import InversionOptimisedGreenhouseGas
from atmospheric_explorer.shapefile import ShapefilesDownloader
from atmospheric_explorer.utils import get_local_folder
from atmospheric_explorer.units_conversion import convert_units_array
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
import zipfile
from glob import glob
import xarray as xr
from datetime import datetime
import geopandas as gdp
from shapely.geometry import mapping
import rioxarray as rio

In [ ]:
#function to move from 0+360 to -180+180 long
def ds_swaplon(ds):
    return ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180)).sortby('longitude')

In [ ]:
manager = InversionOptimisedGreenhouseGas(
    data_variables='carbon_dioxide',
    file_format='zip',
    quantity='surface_flux',
    input_observations='surface',
    time_aggregation='monthly_mean',
    year=[
        '1982', '1983', '1984',
        '1985', '1986', '1987',
        '1988', '1989', '1990',
        '1991', '1992', '1993',
        '1994', '1995', '1996',
        '1997', '1998', '1999',
        '2000', '2001', '2002',
        '2003', '2004', '2005',
        '2006', '2007', '2008',
        '2009', '2010', '2011',
        '2012', '2013', '2014',
        '2015', '2016', '2017',
        '2018', '2019', '2020',
        '2021',
    ],
    month=[
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12'
    ]
)
manager.download()

In [ ]:
files = sorted(list(glob(manager.file_full_path)))

In [ ]:
files

In [ ]:
mm = datetime.strptime(files[0].split('_')[-1].split('.')[0], '%Y%m')
df = xr.open_dataset(files[0])[['flux_foss']]
df = df.expand_dims({'time': [mm]})
for file in files[1:]:
    mm = datetime.strptime(file.split('_')[-1].split('.')[0], '%Y%m')
    temp = xr.open_dataset(file)[['flux_foss']]
    temp = temp.expand_dims({'time': [mm]})
    df = xr.combine_by_coords([df, temp])

In [ ]:
df = df.rio.write_crs('EPSG:4326')

In [ ]:
sh_down = ShapefilesDownloader(
    resolution='10m',
    instance='countries_ita'
)
sh_down.download_shapefile()

In [ ]:
sh_down.shapefiles_name

In [ ]:
sh = gdp.read_file('./italy_shapefile/ne_10m_admin_0_countries_ita.shp', crs='EPSG:4326')

In [ ]:
df_clipped = df.rio.clip(sh[sh['ADMIN'] == 'Italy'].geometry.apply(mapping), sh.crs, drop=True)['flux_foss']

In [ ]:
df_clipped[0].plot()

In [ ]:
df_clipped = df_clipped.where(~df_clipped.isnull(), drop=True)

In [ ]:
df_clipped = df_clipped.sortby('time').mean(dim=['longitude', 'latitude'])

In [ ]:
df_clipped.coords['time.year']

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

In [ ]:
fig.add_trace(
    go.Scatter(
        y=df_clipped.values,
        x=df_clipped.coords['time.year'],
        mode='lines'
    )
)

In [ ]:
# TODO:
## 1 - Shiftare lat e long se necessario -> EAC4 va da 0 a 360
## 2 - Clip paese -> Capire se funziona, dovrebbe funzionare
## 3 - Media annuale -> In realtà ci dovrebbe essere un modo per calcolare il CI su plotly, basta avere diversi valori per anno
## 4 - Plot con CI al 95%

## 5 - le api per scaricare i dati cams e lo shapefile non sembrano funzionare
## 6 - Sistemare il grafico in modo da farlo più simile